In [ ]:
!pip install xlrd
!pip install openpyxl
!pip install pandas
!pip install konlpy
!pip install customized_konlpy
!pip install matplotlib
!pip install emoji
!pip install soynlp

In [ ]:
import pandas as pd
import os
import numpy as np
import re

from emoji import core
import re

from ckonlpy.tag import Twitter
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 전처리 함수
def review_kor(gu): #한국어만 남기기
    file = pd.read_excel(f"/content/drive/MyDrive/playdataproject/data/2024-02-15 '{gu}'.xlsx")
    reviews = file['Review'].dropna(axis = 0)

    removed_emoji_review = []
    for text in reviews :     # 이모지 제거
        review = core.replace_emoji(text, replace = "")
        removed_emoji_review.append(review)

    lst_review = []
    for text in removed_emoji_review :      # 한글, 숫자 남기기
        text = re.sub("[^ㄱ-ㅎ ㅏ-ㅣ가-힣 1-9]","", text) # 정규 표현식 수행
        text = re.sub('^ +', "", text) # 공백은 empty 값으로 변경
        lst_review.append(text)
        .323

    df_review = pd.DataFrame(lst_review)  # dataframe으로 전환
    data = df_review[0].dropna()
    return data

In [ ]:
# 한글만 남은 리뷰 저장
for gu in ['강남구', '서초구', '송파구']:
    review_kor(gu).to_excel(f'preprocessed {gu}.xlsx')
    print("finish")

In [ ]:
# 토큰화 함수
def review_tokenized(gu) :
    file = pd.read_excel(f'/content/drive/MyDrive/playdataproject/data/labeing {gu}.xlsx')
    data = file[['LOCATION', 'Review', 'Label']].dropna(how='any')

    tokenizer = Twitter()
    # 고유명사 등록
    tokenizer.add_dictionary(['콘마요', '분모자', '엽떡', '엽기떡볶이', '떡볶이', '로제', '마라', '주먹김밥', '오리지널', '오리지날', '웨이팅',
                        '맵지리', '쿨피스', '엽기닭볶음탕', '중국당면', '튀김', '만두', '엽기적', '예약', '타지점', '오돌뼈', '짜장엽떡',
                        '어수선', '계란죽', '배달비','배달료', '엽오뎅', '푸짐', '참치마요', '도리탕', '매바매', '점바점', '편리', '사장님',
                        '친절', '엽도리탕', '오더', '맵찔이', '존맛탱', '식자재', '꿔바로우', '강추', '처돌이', '키오스크', '가성비', '주기적',
                        '유부', '맛도리', '찝찝', '재방문', '단짠', '꾸덕', '중독성','역대급', '세트메뉴', '모짜치즈', '포장'], 'Noun')
    # 불용어 등록
    stopwords = ['어요', '떡볶이', '것', '요', '어서','좀', '더', '있다', '같다', '이다', '야하다', '다른', '점', '때', '진짜', '엽떡', '조금'
                '역시', '조금', '별로', '최고', '항상', '자주', '정말', '맛', '지점']

    tokenized_reviews = []
    for review in data['Review']:
        tokenized_sentence = tokenizer.morphs(str(review)) # 토큰화
        stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
        tokenized_reviews.append(stopwords_removed_sentence)
    data['Review'] = tokenized_reviews

    return data

In [ ]:
result1 = review_tokenized('강남구')
print(review_tokenized('강남구').isnull().sum())
result1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 토큰화(형태소tag) 함수
def review_nouns(gu):
    file = pd.read_excel(f'labeing {gu}.xlsx')
    data = file[['Review', 'Label']].dropna(how='any')      # 결측치 제거

    tokenizer = Twitter()
    # 고유명사 등록
    tokenizer.add_dictionary(['콘마요', '분모자', '엽떡', '엽기떡볶이', '떡볶이', '로제', '마라', '주먹김밥', '오리지널', '오리지날', '웨이팅',
                        '맵지리', '쿨피스', '엽기닭볶음탕', '중국당면', '튀김', '만두', '엽기적', '예약', '타지점', '오돌뼈', '짜장엽떡',
                        '어수선', '계란죽', '배달비','배달료', '엽오뎅', '푸짐', '참치마요', '도리탕', '매바매', '점바점', '편리', '사장님',
                        '친절', '엽도리탕', '오더', '맵찔이', '존맛탱', '식자재', '꿔바로우', '강추', '처돌이', '키오스크', '가성비', '주기적',
                        '유부', '맛도리', '찝찝', '재방문', '단짠', '꾸덕', '중독성','역대급', '세트메뉴', '모짜치즈', '포장', '2인분'], 'Noun')

    # 불용어 등록
    stopwords = ['어요', '떡볶이', '것', '요', '어서','좀', '더', '있다', '같다', '이다', '야하다', '다른', '점', '때', '진짜', '엽떡', '조금'
                '역시', '조금', '별로', '최고', '항상', '자주', '정말', '맛', '지점', '말', '느낌']

    tagged_reviews = []
    for review in data['Review'] :
        tagged_sentence = tokenizer.pos(review, stem=True, norm=True,)
        stopwords_removed_word = [(word,tag) for word, tag in tagged_sentence
                                  if (tag == 'Noun' or tag =='Adjective') if not word in stopwords] # 명사, 형용사만 가져와서 불용어 제거
        tagged_reviews.append(stopwords_removed_word)
    data['Review'] = tagged_reviews

    return data

In [ ]:
result2 = review_nouns('강남구')
print(review_tokenized('강남구').isnull().sum())
result2

In [ ]:
result_seocho = review_nouns('서초구')
result_gangnam = review_nouns('강남구')
result_songpa = review_nouns('송파구')

In [ ]:
result_total = pd.concat([result_seocho,result_gangnam,result_songpa], axis = 0)
result_total

In [ ]:
# 긍/부정 리뷰 변수 선언
total_review = result_total['Review']
pos_review = result.query('Label==1')['Review']
neg_review = result.query('Label==0')['Review']

In [ ]:
# 5. 명사&형용사 함께 선별
noun_adj_list = []

for review in pos_review :
    for word, tag in review:
        if tag in ['Noun', 'Adjective']:
            noun_adj_list.append(word)

print(len(noun_adj_list))

In [ ]:
# 5. 명사 혹은 형용사,품사 따로 선별
adj_list = []
noun_list = []

for review in pos_review :
    for word, tag in review:
        if tag in ['Noun']:
            noun_list.append(word)
        elif tag in ['Adjective']:
            adj_list.append(word)

In [ ]:
counts_noun = Counter(noun_list)
counts_adj = Counter(adj_list)
print(counts_noun.most_common(10))
print(counts_adj.most_common(10))

# 여기서부터 시각화
- 단어 빈도수(가로 막대 or 트리맵(점유율))
- 리뷰 길이 비교(히스토그램)
- 키워드 시각화(워드 클라우드)

- 일단 생각한 건 이정도???

In [ ]:
## 나눔폰트 설치 -> 설치 후 런타임 재실행 要

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

# 폰트 변경
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] =False

# tick(눈금)의 음수기호 '-' 가 깨지는 것 처리
mpl.rcParams['axes.unicode_minus'] = False

# 폰트 size변경
mpl.rcParams["font.size"] = 15
# x, y 축의 label 크기 변경
mpl.rcParams['axes.labelsize'] = 12
# x, y 축의 tick(눈금) label 크기 변경
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10

**전체 top 10 키워드**

In [ ]:
counts_words = Counter(noun_adj_list)
top_words = dict(counts_words.most_common(10))
top_words

In [ ]:
# 키워드별 카운트 시각화
import numpy as np

def vis_topwords(top_words):

  y_pos = np.arange(len(top_words))

  plt.grid(alpha = 0.2)
  plt.barh(y_pos, top_words.values(), color = 'k')
  plt.title('Top 20 Keyword Count')
  plt.ylabel("keyword")
  plt.xlabel("횟수")
  plt.yticks(y_pos, top_words.keys())

  # plt.show()

In [ ]:
tw_gn = dict(cnt_noun_adj_gn)
tw_sc = dict(cnt_noun_adj_sc)
tw_sp = dict(cnt_noun_adj_sp)
vis_topwords(tw_gn)
vis_topwords(tw_sc)
vis_topwords(tw_sp)

**점포별 top10 키워드**

In [ ]:
def store_review_top10(구명, 점포명):
    df = eval(f"result_{구명}")

    # 리뷰 데이터 추출
    store_review_list = df.query('LOCATION == @점포명')['Review'].tolist()

    # 리뷰에서 명사와 형용사 추출 후 빈도 계산
    word_counts = Counter(noun_adj(store_review_list))

    # 가장 빈도가 높은 상위 10개의 단어 선택
    top_10_words = dict(word_counts.most_common(10))

    # 시각화
    # vis_topwords() => plt.show() 비활성화 시키기
    vis_topwords(top_10_words)

    # 이미지 저장경로 설정
    저장경로 = '/content/drive/MyDrive/project1/image/강남구'

    # 이미지를 jpg 파일로 저장
    파일경로 = os.path.join(저장경로, f"{점포명}.jpg")
    plt.savefig(파일경로)

In [ ]:
store_review_top10('gangnam', '동대문엽기떡볶이 청담점')

**리뷰 데이터 긍부정 비율**

In [ ]:
# figure 및 서브플롯 설정
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))

# 첫 번째 서브플롯: 'label' 컬럼 값의 수를 막대 그래프로 표시
label_counts = result_seocho['Label'].value_counts()
ax1.bar(label_counts.index, label_counts.values, color=['blue', 'green'])
ax1.set_title('Counts of Label')
ax1.set_xlabel('Seocho')
ax1.set_ylabel('Count')
ax1.set_xticks(label_counts.index)
# ax1.set_xticklabels(['부정', '긍정'])

# 두 번째 서브플롯
label_counts = result_gangnam['Label'].value_counts()
ax2.bar(label_counts.index, label_counts.values, color=['blue', 'green'])
ax2.set_title('Counts of Label')
ax2.set_xlabel('Gangnam')
ax2.set_ylabel('Count')
ax2.set_xticks(label_counts.index)
# ax2.set_xticklabels(['부정', '긍정'])

# 세 번째 서브플롯
label_counts = result_songpa['Label'].value_counts()
ax3.bar(label_counts.index, label_counts.values, color=['blue', 'green'])
ax3.set_title('Counts of Label')
ax3.set_xlabel('Songpa')
ax3.set_ylabel('Count')
ax3.set_xticks(label_counts.index)
# ax3.set_xticklabels(['부정', '긍정'])

# 전체 그래프 표시
plt.tight_layout()
plt.show()


**긍부정 리뷰 길이 비교(히스토그램)**

In [ ]:
### 긍부정 리뷰 길이 비교(히스토그램)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
text_len = pos_review.map(lambda x : len(x))
ax1.hist(text_len, color='red')
ax1.set_title('Positive Reviews')
ax1.set_xlabel('length of samples')
ax1.set_ylabel('number of samples')
print('긍정 리뷰의 평균 길이 :', np.mean(text_len))

text_len = neg_review.map(lambda x : len(x))
ax2.hist(text_len, color='blue')
ax2.set_title('Negative Reviews')
fig.suptitle('Words in texts')
ax2.set_xlabel('length of samples')
ax2.set_ylabel('number of samples')
print('부정 리뷰의 평균 길이 :', np.mean(text_len))

plt.show()

**3구 키워드 비교 Heatmap**

In [ ]:
# 데이터를 딕셔너리로 변환
def list_to_dict(data):
    dictionary = {}
    for key, value in data:
        if key in dictionary:
            dictionary[key] += value
        else:
            dictionary[key] = value
    return dictionary

# 모든 데이터를 딕셔너리로 변환
dict1 = list_to_dict(cnt_noun_adj_gn)
dict2 = list_to_dict(cnt_noun_adj_sc)
dict3 = list_to_dict(cnt_noun_adj_sp)

# 모든 딕셔너리를 하나의 데이터프레임으로 합치기
df_heatmap = pd.DataFrame([dict1, dict2, dict3]).fillna(0).T
df_heatmap.columns = ['강남구', '서초구', '송파구']

# 결과 출력
print(df_heatmap)

df_heatmap = df_heatmap.apply(lambda x : x / np.sum(x))
df_heatmap = df_heatmap.apply(lambda x : (x-min(x)) / (max(x)-min(x)))
df_heatmap = df_heatmap.dropna()

sns.heatmap(df_heatmap, cmap='YlGn')
plt.show()


**워드 클라우드**

In [ ]:
from PIL import Image
icon = Image.open('/content/drive/MyDrive/project1/그릇.png')
plt.imshow(icon)

mask = Image.new("RGB", icon.size, (270,270,270))
mask.paste(icon, icon)
mask = np.array(mask)

In [ ]:
from wordcloud import WordCloud

# WordCloud를 생성한다.
def word_cloud(na_list):

  all_words = [word for word in na_list]

  # Counter 객체를 사용하여 단어의 빈도수를 계산합니다
  counts = Counter(all_words)

  # 가장 많이 등장하는 단어 200개를 추출합니다.
  tags = counts.most_common(200)
  font_path = '/path/to/NanumBarunGothic.ttf'

  wc = WordCloud(font_path=font_path, background_color="white", max_font_size=600, mask=mask, colormap='Reds')
  cloud = wc.generate_from_frequencies(dict(tags))

  plt.figure(figsize=(8, 8))
  plt.axis('off')
  plt.imshow(cloud)
  plt.show()

  # # 생성된 WordCloud를 test.jpg로 보낸다.
  # cloud.to_file(f'wc_{na_list}.jpg')

In [ ]:
word_cloud(seocho_na)

**매장별 긍정 부정 리뷰 비율 (파이차트)**

In [ ]:
def ratio_chart(df_store, store_name):
  wedgeprops = {'width' : 0.75}
  # explode = [0.03] * len(df_store)
  colors = ['#8fd9b6', '#ffc000']

  # store_name에 해당하는 행을 선택
  store_data = df_store.loc[df_store.index == store_name]
  # 'neg'와 'pos' 컬럼의 값을 가져옴
  values = store_data[['neg', 'pos']].values.flatten()
  # 컬럼의 이름을 라벨로 사용
  labels = ['부정', '긍정']
  # 파일명지정
  filename = store_name

  plt.figure(figsize=(10,5))
  plt.pie(values, labels=labels, autopct='%.1f%%', wedgeprops = wedgeprops, shadow=True, colors=colors)
  plt.title('매장별 긍정, 부정 리뷰 비율')

  # 저장경로 = '/content/drive/MyDrive/project1/pie chart'
  # 파일경로 = os.path.join(저장경로, f"{filename}.jpg")
  # plt.savefig(파일경로)

  plt.show()

In [ ]:
import pandas as pd

df_ratio = pd.read_excel("/content/drive/MyDrive/project1/매장별리뷰비율.xlsx")
df_ratio.drop(columns=['Unnamed: 0', 'neg_percent', 'pos_percent'], inplace=True)
df_ratio.set_index(df_ratio.columns[0], inplace=True)
df_ratio

In [ ]:
ratio_chart(df_ratio, '불닭발땡초동대문엽기떡볶이 잠실역점')